## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-21-49-00 +0000,wsj,Opinion | The Government Shutdown Stakes for t...,https://www.wsj.com/opinion/obamacare-subsidie...
1,2025-09-30-21-48-00 +0000,wsj,Opinion | The SEC’s Class Act on Class Actions,https://www.wsj.com/opinion/sec-class-action-l...
2,2025-09-30-21-46-00 +0000,wsj,Opinion | Pete Hegseth and the Generals,https://www.wsj.com/opinion/pete-hegseth-milit...
3,2025-09-30-21-44-56 +0000,nyt,U.N. Security Council Approves Larger Security...,https://www.nytimes.com/2025/09/30/world/ameri...
4,2025-09-30-21-44-00 +0000,wsj,Who Was Michigan Church Shooting Suspect Thoma...,https://www.wsj.com/us-news/who-thomas-jacob-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
137,trump,62
1,shutdown,25
0,government,20
138,deal,19
316,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
282,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,164
49,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...,132
170,2025-09-30-15-10-46 +0000,nypost,Trump gives Hamas ‘3 or 4 days’ to accept his ...,https://nypost.com/2025/09/30/world-news/trump...,128
46,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...,128
267,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,127


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
282,164,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
325,60,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...
46,57,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
126,51,2025-09-30-17-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/jay-schott...
134,48,2025-09-30-16-30-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...
64,41,2025-09-30-19-41-34 +0000,nypost,Inside NYC hoops star’s fall from on-court her...,https://nypost.com/2025/09/30/us-news/inside-n...
12,41,2025-09-30-21-27-55 +0000,nypost,Eric Adams is so anti-Zohran Mamdani he’s warm...,https://nypost.com/2025/09/30/us-news/eric-ada...
49,40,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...
306,35,2025-09-29-23-51-43 +0000,latimes,"California, other states again sue Trump admin...",https://www.latimes.com/politics/story/2025-09...
174,32,2025-09-30-15-02-21 +0000,nypost,Pete Hegseth demands troops meet ‘male-level’ ...,https://nypost.com/2025/09/30/us-news/pete-heg...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
